# Windows en Spark

## Requisitos Previos

Instalar Spark y Java en la Máquina Virtual (VM)

In [1]:
# instalar Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# descargar spark 3.5.0
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [2]:
ls -l # verificar que el .tgz está ahí

total 391016
drwxr-xr-x 1 root root      4096 Nov 17 14:29 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [3]:
# descomprimirlo
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:

!pip install py4j

# Para maps
!pip install folium
!pip install plotly

Definir el entorno

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Iniciar Sesión de Spark (Spark Session)

---

In [7]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# crear la sesión
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [8]:
spark

In [9]:
# Para optimización de conversión a Pandas
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
# Importar funciones sql
from pyspark.sql.functions import *

Descargar y subir los conjuntos de datos (datasets) (DEVELOPERS.CSV, PAYCHECKS.CSV, LEADS.CSV, ROLES.CSV, GAMES.CSV)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving developers.csv to developers.csv
Saving roles.csv to roles.csv
Saving paychecks.csv to paychecks.csv
Saving leads.csv to leads.csv
Saving games.csv to games.csv


## Examples

### Particionado con Windows

In [12]:
developersDF = spark.read.option("header", "true").option("inferSchema", "true").csv("developers.csv")
paychecksDF = spark.read.option("header", "true").option("inferSchema", "true").csv("paychecks.csv")
leadsDF = spark.read.option("header", "true").option("inferSchema", "true").csv("leads.csv")
rolesDF = spark.read.option("header", "true").option("inferSchema", "true").csv("roles.csv")
gamesDF = spark.read.option("inferSchema", "true").option("header", "true").csv("games.csv")

In [13]:
rolesDF.show(2)

+------+---------+----------+----------+
|dev_id|role_name| from_date|   to_date|
+------+---------+----------+----------+
|  1026|   Intern|2016-08-31|9999-01-01|
|  1016|     Lead|2011-11-29|9999-01-01|
+------+---------+----------+----------+
only showing top 2 rows



Rol Más Reciente por Desarrollador

In [14]:
# Filtrar para ver el historial de un desarrollador de ejemplo
# Nota: La columna 'to_date' debe estar en formato YYYY-MM-DD para ordenar correctamente.
# Asumimos que "9999-01-01" es la fecha más lejana, es decir, el rol actual.
rolesDF.filter(col("dev_id") == 1004).show(4)

+------+----------+----------+----------+
|dev_id| role_name| from_date|   to_date|
+------+----------+----------+----------+
|  1004|Senior Dev|2016-06-10|9999-01-01|
|  1004|Junior Dev|2014-04-01|2016-06-10|
|  1004|    Intern|2014-04-01|2014-04-01|
|  1004|Senior Dev|2016-06-10|2020-08-01|
+------+----------+----------+----------+
only showing top 4 rows



In [15]:
from pyspark.sql.functions import col, max, row_number, udf, regexp_replace
from pyspark.sql.window import Window
from pyspark.sql.types import LongType, StringType

In [16]:
# Definición de la Ventana: Particionar por dev_id y ordenar por to_date (descendente)
byDeveloper = Window.partitionBy("dev_id").orderBy(col("to_date").desc())

mostRecentRoleDF = rolesDF.withColumn("datesOrder", row_number().over(byDeveloper)) \
    .filter(col("datesOrder") == 1) \
    .select("dev_id", "role_name", "to_date")

mostRecentRoleDF.filter(col("dev_id") == 1004).show(4)

+------+----------+----------+
|dev_id| role_name|   to_date|
+------+----------+----------+
|  1004|Senior Dev|9999-01-01|
+------+----------+----------+



In [17]:
# Para el ejemplo anterior que vimos en los joins, si utilizamos la partición de ventanas, no es necesario realizar el paso anterior (filtrar la fecha máxima y luego realizar el join sobre la misma tabla).

Los 3 Pagos Máximos por Rol Actual

In [18]:
# 2.1 Unimos los pagos con el rol más reciente (similar a unir salarios y títulos)
# Primero, aseguramos que 'amount' sea LongType (como 'salary' a 'long')
paychecksDF = paychecksDF.withColumn("amount", col("amount").cast(LongType()))

bestPaidPerRoleRawDF = paychecksDF.join(
    mostRecentRoleDF,
    # Unimos por ID de desarrollador y usamos la fecha de finalización del rol como filtro para asegurar que el pago cae en ese rol.
    # Esta es una simplificación del ejemplo original, ya que no tenemos 'from_date' en el DF de pagos.
    paychecksDF.dev_id == mostRecentRoleDF.dev_id,
    "inner"
).drop(mostRecentRoleDF.dev_id).drop("to_date", "pay_date")

print("Pagos unidos a roles recientes:")
bestPaidPerRoleRawDF.show(3)


Pagos unidos a roles recientes:
+------+------+----------+
|dev_id|amount| role_name|
+------+------+----------+
|  1001|139988|Senior Dev|
|  1001|120536|Senior Dev|
|  1001|113190|Senior Dev|
+------+------+----------+
only showing top 3 rows



In [19]:
# 2.2 Aplicamos Window Partitioning
# Definición de la Ventana: Particionar por role_name y ordenar por amount (descendente)
byRole = Window.partitionBy("role_name").orderBy(col("amount").desc())

bestPaidPerRoleDF = bestPaidPerRoleRawDF.withColumn("rank_paycheck", row_number().over(byRole)).filter(col("rank_paycheck") <= 3)

print("Los 3 mejores pagos por rol actual:")
bestPaidPerRoleDF.show(6)

Los 3 mejores pagos por rol actual:
+------+------+----------+-------------+
|dev_id|amount| role_name|rank_paycheck|
+------+------+----------+-------------+
|  1026|114777|    Intern|            1|
|  1022| 96996|    Intern|            2|
|  1022| 96387|    Intern|            3|
|  1046|146399|Junior Dev|            1|
|  1044|145328|Junior Dev|            2|
|  1034|136611|Junior Dev|            3|
+------+------+----------+-------------+
only showing top 6 rows



### UDFs

In [20]:
gamesDF.show(3)

+-------+-----------+-----------------+------------+
|game_id|      title|           engine|release_year|
+-------+-----------+-----------------+------------+
|      1|GameTitle_1|Custom Engine 1.0|      N/A_Q0|
|      2|GameTitle_2|Unreal Engine 5.1|2023_Q_Final|
|      3|GameTitle_3|        Godot 4.1|     2022_Q4|
+-------+-----------+-----------------+------------+
only showing top 3 rows



In [21]:
# Creamos la udf
def is_unreal(engine):
    if "Unreal" in engine:
        return "Yes"
    else:
        return "No"

# Registrar UDF
is_unreal_udf = udf(is_unreal, StringType())

# Usar UDF en el DataFrame
df_with_udf = gamesDF.withColumn("uses_unreal", is_unreal_udf(gamesDF["engine"]))

df_with_udf.show()

+-------+------------+-----------------+------------+-----------+
|game_id|       title|           engine|release_year|uses_unreal|
+-------+------------+-----------------+------------+-----------+
|      1| GameTitle_1|Custom Engine 1.0|      N/A_Q0|         No|
|      2| GameTitle_2|Unreal Engine 5.1|2023_Q_Final|        Yes|
|      3| GameTitle_3|        Godot 4.1|     2022_Q4|         No|
|      4| GameTitle_4|Unreal Engine 5.1|     2024_Q4|        Yes|
|      5| GameTitle_5|     Unity 2023.2|     2023_Q3|         No|
|      6| GameTitle_6|     Unity 2023.2|     2021_Q2|         No|
|      7| GameTitle_7|Unreal Engine 5.1|     2019_Q1|        Yes|
|      8| GameTitle_8|Unreal Engine 5.1|     2020_Q3|        Yes|
|      9| GameTitle_9|Custom Engine 1.0|     2023_Q1|         No|
|     10|GameTitle_10|        Godot 4.1|     2018_Q4|         No|
+-------+------------+-----------------+------------+-----------+



## Ejercicios de Particionado de Windows
1. Rol más reciente por desarrollador: Usando el archivo roles.csv, obtén para cada desarrollador (dev_id) su rol más reciente según from_date.

2. Diferencia de duración de roles por desarrollador: Usando roles.csv, calcula la duración de cada rol en días (to_date - from_date). Luego, calcula la diferencia de duración de cada rol respecto al rol de menor duración del mismo desarrollador.

Ejercicio 1

Ejercicio 2

## Ejercicios UDFs
1. Elige uno de los DF que tenemos, define dos UDF propios y aplícalos (con un withColumn) al DF. Muestra los resultados.